In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append('/content/drive/My Drive/APT/TAR-project-master')

In [3]:
pip install num2words

     |████████████████████████████████| 102kB 5.7MB/s 


In [4]:
import pandas as pd
import numpy as np
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from dataset.preprocessing.preprocessing import word_stem

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
abstracts_path = '/content/drive/My Drive/APT/temp_data/english_covid_abstracts_data.csv'
abstracts = pd.read_csv(abstracts_path)
abstracts.head()

,paper_id,section,text,preprocessed_text
0,86a998617c077f4fe2ab26214995a3548fbc0fc5,Abstract,The recent emergence of the Middle East respir...,the recent emerg middl east respiratori syndro...
1,306ef95a3a91e13a93bcc37fb2c509b67c0b5640,Abstract,Thousands of people in the United States have ...,thousand peopl unit state requir test sars-cov...
2,5e0c586f047ff909c8ed3fe171c8975a90608d08,Abstract,Background: Porcine epidemic diarrhea virus (P...,background porcin epidem diarrhea viru pedv em...
3,1579fbff7af9b156c6f49fee0526e48f852ea460,Abstract,"Currently, live-attenuated IBV vaccines are us...",current live-attenu ibv vaccin use control dis...
4,f6b29be971089bfe0916c64ab9fbddcec38a7436,Abstract,Aims: To determine analytical capabilities of ...,aim determin analyt capabl commonli use faecal...


## Queries

In [0]:
queries = [
           "incubation period of COVID-19",
           "Does smoking increase risks when having covid19?",
           "mortality rate of covid19?",
           "pregnancy complications covid19?"
]

def run_queries(query_engine, queries, k=5):
  results = query_engine.run_query(queries[0], k)
  if len(queries) > 1:
    for q in queries[1:]:
      results = pd.concat([results, query_engine.run_query(q, k)])
  return results

## TF-IDF

In [0]:
from nltk.corpus import stopwords
from query_model.queries import BOWQueryEngine
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from query_model.transformers.bm25 import BM25Transformer

In [0]:
stop_words = set(stopwords.words('english'))
cv = CountVectorizer(stop_words=stop_words)
transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
query_engine = BOWQueryEngine(cv, transformer)
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
# query_engine = BOWQueryEngine.load('/content/drive/My Drive/APT/trained_models/tf_idf.dat')

In [10]:
# query_engine.save('/content/drive/My Drive/APT/trained_models/', 'tf_idf')

Writing object to /content/drive/My Drive/APT/trained_models/tf_idf.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'})

## BM25

In [0]:
from nltk.corpus import stopwords
from query_model.queries import BOWQueryEngine
from sklearn.feature_extraction.text import CountVectorizer
from query_model.transformers.bm25 import BM25Transformer

In [0]:
stop_words = set(stopwords.words('english'))
cv = CountVectorizer(stop_words=stop_words)
transformer = BM25Transformer()
query_engine = BOWQueryEngine(cv, transformer)
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
# query_engine = BOWQueryEngine.load('/content/drive/My Drive/APT/trained_models/bm25.dat')

In [0]:
# query_engine.save('/content/drive/My Drive/APT/trained_models/', 'bm25')

In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'})

## Word2Vec

In [0]:
from query_model.transformers.word2vec import W2VQueryEngine

In [0]:
# TODO train...
query_engine = W2VQueryEngine()
query_engine.fit(abstracts[:10])

In [0]:
# If you want to load already trained model
#query_engine = W2VQueryEngine.load('/content/drive/My Drive/APT/trained_models/w2v.dat')

In [9]:
query_engine.save('/content/drive/My Drive/APT/trained_models/', 'w2v')

Writing object to /content/drive/My Drive/APT/trained_models/w2v.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'})

## Doc2Vec

In [0]:
from query_model.transformers.doc2vec import D2VQueryEngine

In [0]:
# TODO train...
query_engine = D2VQueryEngine()
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
#query_engine = D2VQueryEngine.load('/content/drive/My Drive/APT/trained_models/d2v.dat')

In [14]:
query_engine.save('/content/drive/My Drive/APT/trained_models/', 'd2v')

Writing object to /content/drive/My Drive/APT/trained_models/d2v.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'})

## BERT

In [0]:
pip install -U sentence-transformers

In [0]:
from sentence_transformers import SentenceTransformer
from query_model.queries_BERT import BERTQueryEngine

In [9]:
import torch
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [0]:
query_engine = BERTQueryEngine()
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
query_engine = BERTQueryEngine.load('/content/drive/My Drive/APT/trained_models/BERT.dat')

In [12]:
query_engine.save('/content/drive/My Drive/APT/trained_models/', 'BERT')

Writing object to /content/drive/My Drive/APT/trained_models/BERT.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'})